# LanceDB 임베딩 생성 (RunPod)

RunPod GPU 환경(A100/H100)에서 법령/판례 임베딩을 생성합니다.

## 환경 요구사항
- GPU: RTX 3090/A100/H100 (24GB+ VRAM 권장)
- Template: RunPod PyTorch 2.1+
- Container Disk: 20GB+

## 1. 패키지 설치

In [ ]:
!pip install lancedb sentence-transformers pyarrow ijson psutil tqdm gdown -q
print("패키지 설치 완료")

## 2. 데이터 다운로드

Google Drive에서 데이터를 다운로드합니다.
- `law_cleaned.json` - 법령 데이터 (5,841건)
- `precedents_cleaned.json` - 판례 데이터 (65,107건)

In [ ]:
# Google Drive 파일 ID를 입력하세요
# 공유 -> '링크가 있는 모든 사용자'로 설정 후 FILE_ID 추출
# 예: https://drive.google.com/file/d/ABC123xyz/view -> FILE_ID = ABC123xyz

LAW_FILE_ID = "YOUR_LAW_FILE_ID"  # 법령 JSON
PRECEDENT_FILE_ID = "YOUR_PRECEDENT_FILE_ID"  # 판례 JSON

# 다운로드
# !gdown --id {LAW_FILE_ID} -O law_cleaned.json
# !gdown --id {PRECEDENT_FILE_ID} -O precedents_cleaned.json

# 또는 Jupyter 파일 브라우저에서 직접 업로드
import os
print("현재 파일 목록:")
for f in os.listdir('.'):
    if f.endswith('.json'):
        size_mb = os.path.getsize(f) / (1024 * 1024)
        print(f"  {f}: {size_mb:.1f} MB")

## 3. 스크립트 로드 및 디바이스 확인

In [ ]:
# 스크립트 로드
exec(open('runpod_lancedb_embeddings.py').read())

# 디바이스 정보
device_info, optimal_config = print_device_info()
print(f"\n권장 배치 크기: {optimal_config.batch_size}")

## 4. 임베딩 품질 테스트 (선택)

In [ ]:
# 법률 도메인 임베딩 품질 검증
checker = EmbeddingQualityChecker()
report = checker.quick_test()

## 5. 법령 임베딩 생성

In [ ]:
# 법령 임베딩 (리셋 후 생성)
law_stats = run_law_embedding(
    'law_cleaned.json',
    reset=True,
    # batch_size=128,  # 자동 설정됨
)
print(f"\n법령 임베딩 완료: {law_stats}")

## 6. 판례 임베딩 생성

In [ ]:
# 판례 임베딩 (리셋 없이 추가)
prec_stats = run_precedent_embedding(
    'precedents_cleaned.json',
    reset=False,  # 법령 데이터 유지
    # batch_size=128,  # 자동 설정됨
)
print(f"\n판례 임베딩 완료: {prec_stats}")

## 6-1. 대용량 판례: 분할 처리 (메모리 부족 시)

In [ ]:
# # 판례 분할 (5000건씩)
# split_precedents('precedents_cleaned.json', chunk_size=5000)

# # 분할 파일 전체 처리
# total_stats = run_all_precedent_parts(
#     'precedents_part_*.json',
#     batch_size=64,
# )

## 7. 통계 확인

In [ ]:
show_stats()
print_memory_status()

## 8. 검색 테스트

In [ ]:
import lancedb

# 검색 테스트
model = get_embedding_model()
query = '임대차 보증금 반환'
query_vector = model.encode(query)

db = lancedb.connect('./lancedb_data')
table = db.open_table('legal_chunks')

results = table.search(query_vector).metric('cosine').limit(5).to_pandas()
print(f"\n검색: '{query}'\n")
for _, row in results.iterrows():
    sim = 1 - row['_distance']
    print(f"  [{sim:.4f}] [{row['data_type']}] {row['title'][:50]}")

## 9. 결과 다운로드

In [ ]:
# ZIP 압축
!zip -r lancedb_data.zip ./lancedb_data
import os
size_mb = os.path.getsize('lancedb_data.zip') / (1024 * 1024)
print(f"\nlancedb_data.zip: {size_mb:.1f} MB")
print("파일 브라우저에서 우클릭 -> Download로 다운로드하세요")

In [ ]:
# 메모리 정리
clear_model_cache()
clear_memory()
print("정리 완료")